# Yard Capacity Planning

In [1]:
import pyodbc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import statsmodels.formula.api as smf

C:\Users\PSEN\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Data acquired from SQL server regarding pieces, cubes, employee time by warehouse and department

In [6]:
df.head()

,Shipped Date,Warehouse,Department,Pieces,Cubes
0,2019-05-08,1,1322,23293.000,436954.193
1,2020-01-31,17,4402,8223.000,172705.571
2,2021-01-07,5,3619,19268.000,371035.749
3,2019-05-16,15,3345,15843.957,324392.477
4,2021-02-18,ECR,2642,2195.000,58541.940


In [7]:
df['WHS']=np.where(df['Warehouse']=='1','RKD',
                   np.where(df['Warehouse']=='15','LPT',
                            np.where(df['Warehouse']=='17','ADV',
                                     np.where(df['Warehouse']=='5','RED','ECR'))))

In [8]:
df.head()

,Shipped Date,Warehouse,Department,Pieces,Cubes,WHS
0,2019-05-08,1,1322,23293.000,436954.193,RKD
1,2020-01-31,17,4402,8223.000,172705.571,ADV
2,2021-01-07,5,3619,19268.000,371035.749,RED
3,2019-05-16,15,3345,15843.957,324392.477,LPT
4,2021-02-18,ECR,2642,2195.000,58541.940,ECR


In [10]:
df.dtypes

Shipped Date     object
Warehouse        object
Department       object
Pieces          float64
Cubes           float64
WHS              object
dtype: object

In [11]:
df.drop(['Warehouse'],axis=1,inplace=True)
df.rename(columns={'Shipped Date':'Date','Pieces':'total Unit','Cubes':'total Cube','WHS':'Warehouse'},inplace='True')
df['Date']=pd.to_datetime(df['Date'])
#df['Hour']=df['Hour'].astype('str')

df.dtypes


Date          datetime64[ns]
Department            object
total Unit           float64
total Cube           float64
Warehouse             object
dtype: object

In [15]:

time.head()

,Date,home_wh_id,home_department,Number of Employee
0,2021-06-27,15,3345,52
1,2021-06-27,5,3619,76
2,2021-06-27,17,4402,53
3,2021-06-27,ECR,2642,108
4,2021-06-27,1,1322,55


In [16]:
time['WHS']=np.where(time['home_wh_id']=='1','RKD',
                    np.where(time['home_wh_id']=='15','LPT',
                            np.where(time['home_wh_id']=='17','ADV',
                                    np.where(time['home_wh_id']=='5','RED','ECR'
                                    ))))
                                            

In [17]:
time['Department']=np.where(time['home_wh_id']=='1','1322',
                    np.where(time['home_wh_id']=='15','3345',
                            np.where(time['home_wh_id']=='17','4402',
                                    np.where(time['home_wh_id']=='5','3619','2642'
                                    ))))

In [18]:
#time.drop(['WarehouseID'],axis=1,inplace=True)
time.rename(columns={'WHS':'Warehouse'},inplace='True')

time.dtypes

Date                  object
home_wh_id            object
home_department       object
Number of Employee     int64
Warehouse             object
Department            object
dtype: object

In [19]:

#time.Hour=time.Hour.astype('str')

#time['Hour']=time["Hour"].str.rjust(2,'0')
time.drop(['home_wh_id','home_department'],axis=1,inplace=True)
time['Date']=pd.to_datetime(time['Date'])
time.dtypes

Date                  datetime64[ns]
Number of Employee             int64
Warehouse                     object
Department                    object
dtype: object

In [20]:

time=time[['Date','Warehouse','Department','Number of Employee']]
time.head()

,Date,Warehouse,Department,Number of Employee
0,2021-06-27,LPT,3345,52
1,2021-06-27,RED,3619,76
2,2021-06-27,ADV,4402,53
3,2021-06-27,ECR,2642,108
4,2021-06-27,RKD,1322,55


In [21]:
size=pd.read_excel('square_footage.xlsx')
size

,Warehouse,Department,Size
0,RKD,1322,74234
1,LPT,3345,76369
2,ADV,4402,76467
3,RED,3619,94521
4,ECR,2642,70857
5,MSQ,4005,54600


# Data preprocessing

In [24]:
df1=pd.merge(df,time,on=['Date','Department','Warehouse'], how= "left")

In [31]:
size['key']=size['Warehouse'].astype('str')+size["Department"].astype('str')
size.head()

,Warehouse,Department,Size,key
0,RKD,1322,74234,RKD1322
1,LPT,3345,76369,LPT3345
2,ADV,4402,76467,ADV4402
3,RED,3619,94521,RED3619
4,ECR,2642,70857,ECR2642


In [32]:
df1['key']=df1['Warehouse']+df1['Department']
df1.head()

,Date,Department,total Unit,total Cube,Warehouse,Number of Employee,key
0,2019-05-08,1322,23293.000,436954.193,RKD,153,RKD1322
1,2020-01-31,4402,8223.000,172705.571,ADV,119,ADV4402
2,2021-01-07,3619,19268.000,371035.749,RED,158,RED3619
3,2019-05-16,3345,15843.957,324392.477,LPT,154,LPT3345
4,2021-02-18,2642,2195.000,58541.940,ECR,27,ECR2642


In [33]:
df_final=pd.merge(df1,size[['key','Size']],on=['key'], how='left')
df_final.head()

,Date,Department,total Unit,total Cube,Warehouse,Number of Employee,key,Size
0,2019-05-08,1322,23293.000,436954.193,RKD,153,RKD1322,74234
1,2020-01-31,4402,8223.000,172705.571,ADV,119,ADV4402,76467
2,2021-01-07,3619,19268.000,371035.749,RED,158,RED3619,94521
3,2019-05-16,3345,15843.957,324392.477,LPT,154,LPT3345,76369
4,2021-02-18,2642,2195.000,58541.940,ECR,27,ECR2642,70857


In [34]:
df_final.shape

(5458, 8)

In [37]:
df_final.isnull().sum()

Date                  0
Department            0
total Unit            0
total Cube            0
Warehouse             0
Number of Employee    0
key                   0
Size                  0
dtype: int64

In [39]:
df_final=df_final.dropna(subset=['Number of Employee'])
df_final=df_final.reset_index(drop=True)

In [41]:
df_final.head()

,Date,Department,total Unit,total Cube,Warehouse,Number of Employee,key,Size
0,2019-05-08,1322,23293.000,436954.193,RKD,153,RKD1322,74234
1,2020-01-31,4402,8223.000,172705.571,ADV,119,ADV4402,76467
2,2021-01-07,3619,19268.000,371035.749,RED,158,RED3619,94521
3,2019-05-16,3345,15843.957,324392.477,LPT,154,LPT3345,76369
4,2021-02-18,2642,2195.000,58541.940,ECR,27,ECR2642,70857


In [50]:
df_final.drop(['key'],axis=1,inplace=True)
df_final.head()

,Date,Department,total Unit,total Cube,Warehouse,Number of Employee,Size
0,2019-05-08,1322,23293.000,436954.193,RKD,153,74234
1,2020-01-31,4402,8223.000,172705.571,ADV,119,76467
2,2021-01-07,3619,19268.000,371035.749,RED,158,94521
3,2019-05-16,3345,15843.957,324392.477,LPT,154,76369
4,2021-02-18,2642,2195.000,58541.940,ECR,27,70857


In [62]:
df_final['Year']=df_final['Date'].dt.year
df_final['Month']=df_final['Date'].dt.month
df_final['Week']=df_final['Date'].dt.week
df_final['Day']=df_final['Date'].dt.day
df_final['Week Day']=df_final['Date'].dt.dayofweek
df_final['Year Day']=df_final['Date'].dt.dayofyear
#df_final['Quarter']=df_final['Date'].dt.quarter


C:\Users\PSEN\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [63]:
df_final.head()

,Date,Department,total Unit,total Cube,Warehouse,Number of Employee,Size,Year,Month,Week,Day,Week Day,Year Day
0,2019-05-08,1322,23293.000,436954.193,RKD,153,74234,2019,5,19,8,2,128
1,2020-01-31,4402,8223.000,172705.571,ADV,119,76467,2020,1,5,31,4,31
2,2021-01-07,3619,19268.000,371035.749,RED,158,94521,2021,1,1,7,3,7
3,2019-05-16,3345,15843.957,324392.477,LPT,154,76369,2019,5,20,16,3,136
4,2021-02-18,2642,2195.000,58541.940,ECR,27,70857,2021,2,7,18,3,49


In [64]:
df_final.sort_values(by='Date', ascending= True, inplace= True)

In [65]:
df_final.drop(['Date','Department'],axis=1,inplace=True)
df_final=df_final[['Year','Month','Day','Year Day','Week Day','Warehouse','total Unit','total Cube','Size','Number of Employee']]

In [66]:
df_final.head()

,Year,Month,Day,Year Day,Week Day,Warehouse,total Unit,total Cube,Size,Number of Employee
1101,2018,6,28,179,3,ECR,12661.000,285028.367,70857,114
2722,2018,6,28,179,3,RED,11666.000,223682.705,94521,139
1462,2018,6,28,179,3,LPT,12636.976,263363.623,76369,140
2182,2018,6,28,179,3,RKD,16961.002,329403.893,74234,141
2362,2018,6,28,179,3,ADV,9025.001,158894.347,76467,80


In [67]:
#df_final=pd.concat([df_final, pd.get_dummies(df_final['Hour'])], axis=1)
#df_final.drop(['Hour'],axis=1,inplace=True)
df_final=pd.concat([df_final, pd.get_dummies(df_final['Warehouse'])], axis=1)
df_final.drop(['Warehouse'],axis=1,inplace=True)
#df_final=pd.concat([df_final, pd.get_dummies(df_final['Department'])], axis=1)
#df_final.drop(['Department'],axis=1,inplace=True)
df_final.head()

,Year,Month,Day,Year Day,Week Day,total Unit,total Cube,Size,Number of Employee,ADV,ECR,LPT,RED,RKD
1101,2018,6,28,179,3,12661.000,285028.367,70857,114,0,1,0,0,0
2722,2018,6,28,179,3,11666.000,223682.705,94521,139,0,0,0,1,0
1462,2018,6,28,179,3,12636.976,263363.623,76369,140,0,0,1,0,0
2182,2018,6,28,179,3,16961.002,329403.893,74234,141,0,0,0,0,1
2362,2018,6,28,179,3,9025.001,158894.347,76467,80,1,0,0,0,0


In [70]:
df_final.head()

,Year,Month,Day,Year Day,Week Day,total Unit,total Cube,Size,Number of Employee,ADV,ECR,LPT,RED,RKD
1101,2018,6,28,179,3,12661.000,285028.367,70857,114,0,1,0,0,0
2722,2018,6,28,179,3,11666.000,223682.705,94521,139,0,0,0,1,0
1462,2018,6,28,179,3,12636.976,263363.623,76369,140,0,0,1,0,0
2182,2018,6,28,179,3,16961.002,329403.893,74234,141,0,0,0,0,1
2362,2018,6,28,179,3,9025.001,158894.347,76467,80,1,0,0,0,0


# Model Development and evaluation

In [72]:
y=df_final['Number of Employee']
x=df_final.drop(['Number of Employee'],axis=1)

In [73]:
x.head()

,Year,Month,Day,Year Day,Week Day,total Unit,total Cube,Size,ADV,ECR,LPT,RED,RKD
1101,2018,6,28,179,3,12661.000,285028.367,70857,0,1,0,0,0
2722,2018,6,28,179,3,11666.000,223682.705,94521,0,0,0,1,0
1462,2018,6,28,179,3,12636.976,263363.623,76369,0,0,1,0,0
2182,2018,6,28,179,3,16961.002,329403.893,74234,0,0,0,0,1
2362,2018,6,28,179,3,9025.001,158894.347,76467,1,0,0,0,0


In [74]:
from sklearn.model_selection import TimeSeriesSplit
tss=TimeSeriesSplit(n_splits=10)


In [75]:
from sklearn.ensemble import RandomForestRegressor

regressor=RandomForestRegressor(n_estimators=300,random_state=0)


In [76]:



from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from statistics import mean
result=[]

tss=TimeSeriesSplit(n_splits=10)
regressor=RandomForestRegressor(n_estimators=300,random_state=0)
for train_index, test_index in tss.split(x):
    x_train, x_test=x.iloc[train_index, :].values, x.iloc[test_index, :].values
    y_train, y_test=y.iloc[train_index].values, y.iloc[test_index].values
    regressor.fit(x_train,y_train)
    prediction=(regressor.predict(x_test))
    actual=(y_test)
    MAPE=np.mean(np.abs((actual-prediction)/actual))*100
    result.append(MAPE)
    
print(result)
print('Min MAPE:',min(result))
print('Average MAPE:',mean(result))
print('Max MAPE:',max(result))


[13.038210002480053, 11.880171115011494, 11.02275150764278, 9.524023696765006, 14.055829245568665, 144.35452205528773, 34.04259113380298, 39.183174656510936, 20.240318952614526, 11.420715203706989]
Min MAPE: 9.524023696765006
Average MAPE: 30.876230756939115
Max MAPE: 144.35452205528773


In [77]:
regressor.fit(x,y)

RandomForestRegressor(n_estimators=300, random_state=0)

In [78]:
df_final_imp=df_final.drop(['Number of Employee'],axis=1)
importance=regressor.feature_importances_
importance_features=list(zip(importance,df_final_imp.columns))
importance_features.sort(reverse=True)

In [79]:
importance_features

[(0.5421044222387973, 'Week Day'),
 (0.12678441333031, 'total Cube'),
 (0.10348558896912474, 'total Unit'),
 (0.07835835502765354, 'Year Day'),
 (0.04405836936349344, 'Year'),
 (0.035965992463458885, 'Size'),
 (0.016861053652066613, 'ECR'),
 (0.013402126105530744, 'Day'),
 (0.010264194362855637, 'LPT'),
 (0.009089017628126217, 'RED'),
 (0.007021917541636951, 'Month'),
 (0.006936603497860818, 'ADV'),
 (0.005667945819085085, 'RKD')]